In [1]:
# ============================================
# Imports and Setup
# ============================================
import pandas as pd
import numpy as np
from math import log, ceil, floor, log2, exp
from collections import defaultdict

# ============================================
# Data Preprocessing Function
# ============================================
def load_and_prepare_data(filepath):
    df = pd.read_csv(filepath)
    df = df.sort_values(by="trial")

    arm_pulls = defaultdict(list)
    for _, row in df.iterrows():
        arm = int(row["arm"])
        arm_pulls[arm].append((row["reward"], row["cost"]))

    return df, arm_pulls

# ============================================
# Confidence Estimator
# ============================================
def hoeffding_confidence(chosen_arm, sample_means, num_pulls):
    chosen_mean = sample_means.get(chosen_arm, 0)
    confidences = []
    for arm, mean in sample_means.items():
        if arm == chosen_arm:
            continue
        delta = chosen_mean - mean
        if delta <= 0:
            confidences.append(0.0)
        else:
            n = min(num_pulls.get(arm, 1), num_pulls.get(chosen_arm, 1))
            conf = 1 - exp(-0.5 * n * delta**2)
            confidences.append(conf)
    return min(confidences) if confidences else 1.0

# ============================================
# Exponential-Gap Elimination (Fixed Confidence)
# ============================================
def exponential_gap_elimination(arm_pulls, delta=0.05):
    S = list(arm_pulls.keys())
    r = 1
    total_cost = 0
    total_pulls = 0
    sample_means = {}
    num_pulls = {}

    while len(S) > 1:
        ε_r = 2 ** (-r / 4)
        δ_r = delta / (50 * r ** 3)
        t_r = ceil((2 / (ε_r ** 2)) * log(2 / δ_r))

        max_available = min([len(arm_pulls[arm]) - num_pulls.get(arm, 0) for arm in S])
        if max_available == 0:
            print(f"Stopping early: no data left to sample in round {r}")
            break
        t_r = min(t_r, max_available)

        round_means = {}
        valid_arms = []

        for arm in S:
            start = num_pulls.get(arm, 0)
            available = len(arm_pulls[arm]) - start
            arm_tr = min(t_r, available)
            if arm_tr <= 0:
                continue

            pulls = arm_pulls[arm][start:start + arm_tr]
            rewards = [min(max(r, 0.0), 1.0) for r, _ in pulls]  # Bounded normal rewards
            costs = [c for _, c in pulls]

            if not rewards:
                continue

            avg = np.mean(rewards)
            sample_means[arm] = avg
            round_means[arm] = avg
            num_pulls[arm] = start + len(rewards)
            total_cost += sum(costs)
            total_pulls += len(rewards)
            valid_arms.append(arm)

        if len(valid_arms) == 0:
            print(f"Stopping early: no arms with data in round {r}")
            break

        best_arm = max(round_means, key=round_means.get)
        p_hat_star = round_means[best_arm]
        S = [arm for arm in valid_arms if round_means[arm] >= p_hat_star - ε_r]
        r += 1

    chosen_arm = S[0] if S else None
    confidence = hoeffding_confidence(chosen_arm, sample_means, num_pulls)
    return chosen_arm, total_pulls, total_cost, confidence

# ============================================
# Sequential Halving (Fixed Budget)
# ============================================
def sequential_halving(arm_pulls, total_budget):
    S = list(arm_pulls.keys())
    R = ceil(log2(len(S)))
    budget_used = 0
    total_pulls = 0
    sample_means = {}
    num_pulls = {}

    for r in range(R):
        if len(S) <= 1:
            break

        avg_costs = {arm: np.mean([c for _, c in arm_pulls[arm]]) for arm in S}
        mean_cost_per_arm = sum(avg_costs.values()) / len(S)
        max_total_pulls = floor((total_budget - budget_used) / mean_cost_per_arm)
        t_r = max(floor(max_total_pulls / len(S)), 1)

        max_available = min([len(arm_pulls[arm]) - num_pulls.get(arm, 0) for arm in S])
        if max_available == 0:
            print(f"Stopping early: no data left to sample in round {r}")
            break
        t_r = min(t_r, max_available)

        round_means = {}
        valid_arms = []

        for arm in S:
            start = num_pulls.get(arm, 0)
            available = len(arm_pulls[arm]) - start
            arm_tr = min(t_r, available)
            if arm_tr <= 0:
                continue

            pulls = arm_pulls[arm][start:start + arm_tr]
            rewards = [min(max(r, 0.0), 1.0) for r, _ in pulls]  # Bounded normal rewards
            costs = [c for _, c in pulls]
            cost_sum = sum(costs)

            if not rewards or (budget_used + cost_sum > total_budget):
                continue

            sample_means[arm] = np.mean(rewards)
            round_means[arm] = sample_means[arm]
            num_pulls[arm] = start + len(rewards)
            budget_used += cost_sum
            total_pulls += len(rewards)
            valid_arms.append(arm)

        if len(valid_arms) == 0:
            print(f"Stopping early: no arms with data in round {r}")
            break

        sorted_arms = sorted(round_means.items(), key=lambda x: x[1], reverse=True)
        S = [arm for arm, _ in sorted_arms[:ceil(len(valid_arms) / 2)]]

        if budget_used >= total_budget:
            break

    chosen_arm = S[0] if S else None
    confidence = hoeffding_confidence(chosen_arm, sample_means, num_pulls)
    return chosen_arm, total_pulls, budget_used, confidence

# ============================================
# Evaluation Function
# ============================================
def run_algorithms_on_dataset(filepath, delta=0.05, budget=500):
    print(f"\n--- Running on dataset: {filepath.split('/')[-1]} ---")
    df, arm_pulls = load_and_prepare_data(filepath)

    print("Running Exponential-Gap Elimination...")
    ege_arm, ege_pulls, ege_cost, ege_conf = exponential_gap_elimination(arm_pulls, delta=delta)

    print("Running Sequential Halving...")
    sh_arm, sh_pulls, sh_cost, sh_conf = sequential_halving(arm_pulls, total_budget=budget)

    true_means = df.groupby("arm")["reward"].mean().to_dict()
    best_true_arm = max(true_means, key=true_means.get)

    results = {
        'ege': {
            'pulls': ege_pulls,
            'cost': ege_cost,
            'confidence': ege_conf,
            'correct': ege_arm == best_true_arm
        },
        'sh': {
            'pulls': sh_pulls,
            'cost': sh_cost,
            'confidence': sh_conf,
            'correct': sh_arm == best_true_arm
        }
    }

    return results
